In [11]:
import pandas as pd

In [12]:
# def read_tbl():
#     df = pd.read_excel("../data/input/Invisible_Institue_Public_Records_Request_Response.xlsx")

#     df = df.rename(columns={"ACADEMYID": "uid", 
#                             "NAME": "name", "ACTIONEFFECTIVEDATE": 
#                             "action_date", "ACTIONSTATUS": "action_status",
#                             "CERTIFICATION": "agency_name", 
#                             "ISSUED": "certification_issue_date", 
#                             "CURRENTCERTSTATUS": "current_certificatioan_status", 
#                             "AGENCYORI": "agency_uid", 
#                             "ACTIONTYPE": "action_type",
#                             "TITLE": "rank", 
#                             "CERTIFICATIONTYPE": "certification_type",
#                             })


#     return df 

# def transform_employment_history(df):    
#     df['action_date'] = pd.to_datetime(df['action_date'])
    
#     # Create a function to determine if a row represents a separation
#     def is_separation(status):
#         return 'Separated' in str(status)
    
#     employment_stints = []
    
#     for (name, uid, cert), group in df.groupby(['name', 'uid', 'agency_name']):
#         group = group.sort_values('action_date')
        
#         # If there's only one row, handle it separately
#         if len(group) == 1:
#             row = group.iloc[0]
#             stint_record = row.copy()
#             stint_record['start_date'] = row['action_date']
#             # If it's a separation, use the same date as end_date
#             stint_record['end_date'] = row['action_date'] if is_separation(row['action_status']) else None
#             employment_stints.append(stint_record)
#             continue
            
#         separation_rows = group[group['action_status'].apply(is_separation)]
        
#         if len(separation_rows) > 0:
#             # Handle multiple separations for same agency
#             for idx, separation_row in separation_rows.iterrows():
#                 # Find all rows before this separation
#                 mask = (group['action_date'] <= separation_row['action_date'])
#                 stint_rows = group[mask]
                
#                 if len(stint_rows) > 0:  # Add check to ensure we have rows
#                     # Get the earliest action_date as start_date
#                     start_date = stint_rows['action_date'].min()
#                     end_date = separation_row['action_date']
                    
#                     # Create employment stint record using the most recent row's data
#                     stint_record = stint_rows.iloc[-1].copy()
#                     stint_record['start_date'] = start_date
#                     stint_record['end_date'] = end_date
#                     employment_stints.append(stint_record)
                
#                 # Remove processed rows for next iteration
#                 group = group[~mask]
        
#         # Handle remaining rows (current employment with no separation)
#         if len(group) > 0:
#             stint_record = group.iloc[-1].copy()
#             stint_record['start_date'] = group['action_date'].min()
#             stint_record['end_date'] = None
#             employment_stints.append(stint_record) 

#     if not employment_stints: 
#         return pd.DataFrame(columns=df.columns.tolist() + ['start_date', 'end_date'])
        
#     result = pd.DataFrame(employment_stints)
    
#     cols = ['start_date', 'end_date'] + [col for col in result.columns 
#                                         if col not in ['start_date', 'end_date']]
#     result = result[cols]
    
#     return result


# def split_name(fullname):
#     # Split on comma to separate last name from the rest
#     parts = fullname.split(',')
#     last_name = parts[0].strip()
    
#     # Handle the first/middle/suffix part
#     first_middle_suffix = parts[1].strip().split()
    
#     # Get first name (first word after comma)
#     first_name = first_middle_suffix[0]
    
#     # Initialize middle name and suffix as empty strings
#     middle_name = ''
#     suffix = ''
    
#     # If we have more than just the first name
#     if len(first_middle_suffix) > 1:
#         # Check if the last word is a common suffix
#         common_suffixes = {'jr', 'jr.', 'sr', 'sr.', 'ii', 'iii', 'iv'}
#         last_word = first_middle_suffix[-1].lower()
        
#         if last_word in common_suffixes:
#             # If there are words between first name and suffix, they form the middle name
#             if len(first_middle_suffix) > 2:
#                 middle_name = ' '.join(first_middle_suffix[1:-1])
#             suffix = first_middle_suffix[-1]
#         else:
#             # If no suffix, all words after first name form the middle name
#             middle_name = ' '.join(first_middle_suffix[1:])
    
#     return first_name, middle_name, last_name, suffix

# def fix_dates(df):
#     df.loc[:, "action_date"] = pd.to_datetime(df.action_date, errors="coerce")
#     return df 

# df = read_tbl()

# # actiontype: issue import references a start_date, actiontype update[import]  and action_status separated indicates an end_date



# def normalize_cols(df):
#     df.loc[:, "end_date"] = df.end_date.astype(str).str.replace(r"NaT", "", regex=True)
#     df.loc[:, "first_name"] = df.first_name.str.lower()
#     df.loc[:, "middle_name"] = df.middle_name.str.lower()
#     df.loc[:, "last_name"] = df.last_name.str.lower()
#     df.loc[:, "suffix"] = df.suffix.str.lower()
#     return df 


# df = df.pipe(fix_dates).pipe(transform_employment_history)


# df[['first_name', 'middle_name', 'last_name', 'suffix']] = df['name'].apply(lambda x: pd.Series(split_name(x)))

# df = df.pipe(normalize_cols)

# df = df[["start_date", "end_date", "uid", "agency_name", "rank", "first_name", "middle_name", "last_name", "suffix"]]
 


# # df.to_csv("../data/output/north-carolina-processed.csv", index=False)

In [13]:
df = pd.read_csv("../data/input/nc-mid.csv")

df

,start_date,end_date,uid,agency_name,rank,first_name,middle_name,last_name,suffix
0,1999-12-01,NaN,0012-1559,Asheville Police Department - Law Enforcement ...,Lieutenant,sean,thomas,aardema,NaN
1,1979-11-02,2005-05-01,0012-1548,Asheville Police Department - Law Enforcement ...,Deputy Sheriff,thomas,frederick,aardema,NaN
2,2001-04-12,NaN,0012-1548,Buncombe County Sheriff's Office - Deputy Sheriff,Deputy Sheriff,thomas,frederick,aardema,NaN
3,2011-05-17,2012-06-28,0012-1548,Marion Police Department - Law Enforcement Off...,Deputy Sheriff,thomas,frederick,aardema,NaN
4,2019-12-19,NaN,2944-5526,Butner Public Safety - Law Enforcement Officer,Public Safety Officer,justin,wayne,aaron,NaN
...,...,...,...,...,...,...,...,...,...
122868,1998-12-07,NaN,0700-8529,Burke County Sheriff's Office - Deputy Sheriff,Patrolman,david,c,warlick,NaN
122869,1998-07-31,2004-05-17,0700-8529,Long View Police Department - Law Enforcement ...,Patrolman,david,c,warlick,NaN
122870,2005-05-17,2022-09-21,0700-8529,NC Dept of Revenue-Unauthorized Substance Tax ...,Patrolman,david,c,warlick,NaN
122871,2022-09-06,NaN,6513-0135,Pamlico County Sheriff's Office - Deputy Sheriff,Detention Officer,brian,NaN,wielhouwer,NaN


In [14]:
df.sort_values("uid")

,start_date,end_date,uid,agency_name,rank,first_name,middle_name,last_name,suffix
122721,2009-06-29,NaN,****0019-8088,Kannapolis Police Department - Law Enforcement...,Officer,david,mark,zienka,ii
63259,2009-11-25,NaN,*0097-2608,Wake County Sheriff's Office - Deputy Sheriff,Officer,charles,eugene,lambert,NaN
63260,1983-05-19,1983-08-31,*0097-2608,Wake County Sheriff's Office - Law Enforcement...,Officer,charles,eugene,lambert,NaN
63257,2019-02-19,NaN,*0097-2608,NC State Capitol Police - Law Enforcement Officer,Officer,charles,eugene,lambert,NaN
63255,1999-09-14,2001-09-15,*0097-2608,Clayton Police Department - Law Enforcement Of...,Officer,charles,eugene,lambert,NaN
...,...,...,...,...,...,...,...,...,...
82093,1981-07-23,1988-12-14,9998-8200,Franklinton Police Department - Law Enforcemen...,NaN,jimmy,winston,oneal,NaN
37268,1998-11-23,2000-05-12,9998-8909,Wilson County Sheriff's Office - Deputy Sheriff,NaN,chad,holmes,fowler,NaN
37267,1998-07-09,1998-10-13,9998-8909,Richlands Police Department - Law Enforcement ...,NaN,chad,holmes,fowler,NaN
121523,1987-06-10,1987-11-12,9998-9251,Woodfin Police Department - Law Enforcement Of...,NaN,alan,kenneth,wren,NaN
